In [2]:
import pandas as pd
import nltk as nltk
import texthero as hero
from sklearn.model_selection import train_test_split

import re
from nltk.tokenize import word_tokenize
import gensim
import string
from nltk.probability import FreqDist
from tensorflow.keras.optimizers import Adam


2022-05-13 14:20:41.385797: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-13 14:20:41.385822: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
df = pd.read_csv('text_emotion.csv')

dfTweet = df.drop(['tweet_id','author'],axis = 1)
dfTweet=dfTweet.applymap(str.lower)

dfJournal = pd.read_csv('Emotion_final.csv')
dfJournal=dfJournal.applymap(str.lower)


In [4]:

dfTweet.rename(columns={'sentiment': 'Emotion', 
                           'content': 'Text'}, inplace=True)
#dfTweet[dfTweet['index']==14461]
dfTweet.iloc[[14461]]                

,Emotion,Text
14461,worry,"ï¿½ miss my future, ï¿½ wanna see her ï¿½ for..."


In [5]:
dfTweet.Emotion.value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: Emotion, dtype: int64

In [6]:
dfJournal.Emotion.value_counts()

happy       7029
sadness     6265
anger       2993
fear        2652
love        1641
surprise     879
Name: Emotion, dtype: int64

In [7]:
# dfJournal.Emotion.replace({'happy': 'happiness'}, regex=True,inplace=True)
# dfJournal.Emotion.replace({'enthusiasm': 'happiness'}, regex=True,inplace=True)
# dfJournal.Emotion.replace({'fun': 'happiness'}, regex=True,inplace=True)
# dfJournal.Emotion.replace({'relief': 'neutral'}, regex=True,inplace=True)
# dfJournal.Emotion.replace({'empty': 'neutral'}, regex=True,inplace=True)
# dfJournal.Emotion.replace({'boredom': 'neutral'}, regex=True,inplace=True)
# dfJournal.Emotion.replace({'hate': 'sadness'}, regex=True,inplace=True)
# dfJournal.Emotion.replace({'worry': 'surprise'}, regex=True,inplace=True)
# dfJournal.Emotion.replace({'fear': 'anger'}, regex=True,inplace=True)

# dfJournal = dfJournal.drop(dfJournal[dfJournal.Emotion == 'surprise'].index)
dfJournal = dfJournal.drop(dfJournal[dfJournal.Emotion == 'surprise'].index)
# dfJournal

dfJournal.Emotion.replace({'happy': 0}, regex=True,inplace=True)
dfJournal.Emotion.replace({'sadness': 1}, regex=True,inplace=True)
dfJournal.Emotion.replace({'anger': 2}, regex=True,inplace=True)
dfJournal.Emotion.replace({'fear': 3}, regex=True,inplace=True)
dfJournal.Emotion.replace({'love': 4}, regex=True,inplace=True)
nb_labels = len(dfJournal.Emotion.value_counts())

In [8]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop = list(set(stop) - set(['not','no']))
lemmatizer = nltk.stem.WordNetLemmatizer()

def tweets_cleaning(tweet, remove_emojis=True):
    """Apply function to a clean a tweet"""
    tweet = tweet.lower().strip()
    # romove urls
    url = re.compile(r'https?://\S+|www\.\S+')
    tweet = url.sub(r'',tweet)
    # remove html tags
    html = re.compile(r'<.*?>')
    tweet = html.sub(r'',tweet)
    # remove punctuation
    operator = str.maketrans('','',string.punctuation) #????
    tweet = tweet.translate(operator)
    if remove_emojis:
        tweet = tweet.encode('ascii', 'ignore').decode('utf8').strip()
    lambda tweet: re.sub(r'https?:\/\/\S+', '', tweet)
    lambda tweet: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', tweet)
    tweet=re.sub(r'@[A-Za-z0-9]+'," ",tweet) ##Removing the usernames
    tweet=re.sub(r'^[A-Za-z0-9.!?]+'," ",tweet) ##Removing digits and punctuations
    tweet=re.sub(r'https?://[A-Za-z0-9./]+'," ",tweet) ## removing links
    tweet=re.sub(r' +'," ",tweet)
    tweet = tweet.lower()
    tweet = re.sub(r"\'s", " ", tweet)
    tweet = re.sub(r"\'ve", " have ", tweet)
    tweet = re.sub(r"can't", "cannot ", tweet)
    tweet = re.sub(r"n't", " not ", tweet)
    tweet = re.sub(r"\'d", " would ", tweet)
    tweet = re.sub(r"\'ll", " will ", tweet)
    tweet = re.sub(r"\'scuse", " excuse ", tweet)
    tweet = tweet.strip(' ')
    tweet = tweet.strip('. .')
    tweet = tweet.replace('.',' ')
    tweet = tweet.replace('-',' ')
    tweet = tweet.replace("’", "'").replace("′", "'").replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")
    tweet = tweet.replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")
    tweet = tweet.replace("€", " euro ").replace("'ll", " will")
    tweet = tweet.replace("don't", "do not").replace("didn't", "did not").replace("im","i am").replace("it's", "it is")
    tweet = tweet.replace(",000,000", "m").replace("n't", " not").replace("what's", "what is")
    tweet = tweet.replace(",000", "k").replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")
    tweet = tweet.replace("he's", "he is").replace("she's", "she is").replace("'s", " own")
    tweet = tweet.replace("didnt","did not")
    tweet = re.sub('\s+', ' ', tweet)
    tweet=tweet.split()
    tweet=[lemmatizer.lemmatize(word) for word in tweet if word not in stop]
    tweet=' '.join(word for word in tweet)
    return tweet.lower()

In [9]:
df = dfJournal
df['clean_text'] = df['Text'].apply(tweets_cleaning)

train_data=df.iloc[:,-1].values
df['clean_text']=train_data
df

,Text,Emotion,clean_text
0,i didnt feel humiliated,1,not feel humiliated
1,i can go from feeling so hopeless to so damned...,1,go feeling hopeless damned hopeful around some...
2,im grabbing a minute to post i feel greedy wrong,2,grabbing minute post feel greedy wrong
3,i am ever feeling nostalgic about the fireplac...,4,ever feeling nostalgic fireplace know still pr...
4,i am feeling grouchy,2,feeling grouchy
...,...,...,...
21454,melissa stared at her friend in dism,3,stared friend dism
21455,successive state elections have seen the gover...,3,state election seen governing party pummelled ...
21456,vincent was irritated but not dismay,3,irritated not dismay
21457,kendall-hume turned back to face the dismayed ...,3,turned back face dismayed coup


In [10]:
import numpy as np


In [11]:
idx=np.random.randint(0,len(df),1200)
Y_array = np.array(df["Emotion"])
X_array = np.array(df['clean_text'])


X_test=X_array[idx[0:1000]]
y_test=Y_array[idx[0:1000]]
X_virgin = X_array[idx[1001:1200]]
y_virgin = Y_array[idx[1001:1200]]
X_train=np.delete(X_array,idx,axis=0)
y_train=np.delete(Y_array,idx,axis=0)

In [12]:
y_test

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 4, 1, 0, 0, 0, 1, 0, 4, 0, 0, 0,
       4, 1, 1, 0, 0, 0, 0, 2, 1, 0, 1, 2, 0, 3, 0, 4, 2, 0, 1, 1, 3, 1,
       1, 1, 3, 1, 0, 2, 0, 4, 2, 0, 3, 1, 1, 2, 1, 1, 2, 0, 1, 2, 3, 0,
       1, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 1, 0, 3, 0, 0, 0, 1, 4, 0, 2, 1,
       1, 3, 1, 1, 2, 1, 2, 3, 0, 4, 2, 0, 3, 2, 1, 0, 0, 2, 2, 1, 1, 1,
       4, 2, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 2, 1, 1, 1, 0, 2, 0, 2, 0,
       0, 3, 3, 3, 2, 1, 0, 1, 3, 3, 4, 1, 0, 0, 0, 2, 0, 1, 1, 0, 0, 2,
       1, 0, 4, 0, 3, 4, 1, 2, 4, 2, 1, 1, 4, 1, 1, 0, 0, 0, 0, 1, 4, 0,
       0, 0, 0, 2, 1, 0, 1, 0, 1, 0, 4, 0, 1, 3, 1, 0, 2, 2, 2, 2, 0, 0,
       4, 4, 1, 4, 2, 1, 0, 0, 3, 1, 3, 2, 3, 3, 3, 0, 0, 1, 2, 1, 1, 1,
       0, 4, 3, 1, 4, 0, 2, 1, 1, 2, 0, 0, 0, 1, 1, 0, 1, 0, 3, 1, 1, 4,
       2, 3, 2, 0, 4, 0, 0, 0, 3, 0, 0, 2, 1, 1, 3, 0, 3, 0, 2, 0, 4, 0,
       1, 1, 3, 0, 1, 4, 1, 1, 2, 1, 1, 4, 0, 0, 0, 1, 1, 2, 2, 1, 3, 3,
       0, 4, 1, 3, 1, 3, 2, 1, 1, 3, 1, 2, 0, 4, 0,

# Vectorizer

In [13]:
from tensorflow.keras.layers import TextVectorization
import tensorflow as tf
vectorizer = TextVectorization(max_tokens=40000,output_mode='int', output_sequence_length=35,standardize='lower_and_strip_punctuation')





2022-05-13 14:20:48.687158: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-13 14:20:48.687205: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (PC-TheophileZ): /proc/driver/nvidia/version does not exist
2022-05-13 14:20:48.687512: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
vectorizer.adapt(X_train)
X_train

array(['not feel humiliated',
       'go feeling hopeless damned hopeful around someone care awake',
       'grabbing minute post feel greedy wrong', ...,
       'irritated not dismay', 'turned back face dismayed coup',
       'dismayed not surpris'], dtype=object)

In [15]:
len(vectorizer.get_vocabulary())

16040

In [16]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [17]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip

In [18]:
path_to_glove_file = "glove.twitter.27B.100d.txt"
import numpy as np

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 1193514 word vectors.


In [19]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 14486 words (1554 misses)


In [20]:
X_train

array(['not feel humiliated',
       'go feeling hopeless damned hopeful around someone care awake',
       'grabbing minute post feel greedy wrong', ...,
       'irritated not dismay', 'turned back face dismayed coup',
       'dismayed not surpris'], dtype=object)

# Model

In [50]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    #embeddings_initializer = "uniform",
    trainable=False,
)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, restore_best_weights=True)

#int_sequences_input = tf.keras.Input(shape=(None,), dtype="int64")

input = tf.keras.Input(shape=(1,), dtype='string')

x = vectorizer(input)
#x = layers.Dropout(0.2)(x)
x = embedding_layer(x)
x = layers.Dropout(0.2)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)
preds = layers.Dense(nb_labels, activation="softmax")(x)
model = tf.keras.Model(input, preds)
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 35)               0         
 torization)                                                     
                                                                 
 embedding_2 (Embedding)     (None, 35, 100)           1604200   
                                                                 
 dropout_10 (Dropout)        (None, 35, 100)           0         
                                                                 
 conv1d_6 (Conv1D)           (None, 35, 128)           12928     
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 7, 128)           0         
 1D)                                                       

In [51]:
input = tf.keras.Input(shape=(1,), dtype='string')

x = vectorizer(input)
#x = layers.Dropout(0.2)(x)
x = embedding_layer(x)
x = layers.Dropout(0.2)(x)
x = layers.Bidirectional(tf.keras.layers.LSTM(100,return_sequences=True))(x)
x = layers.Bidirectional(tf.keras.layers.LSTM(100))(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.3)(x)
preds = layers.Dense(nb_labels, activation="softmax")(x)
modelLSTM = tf.keras.Model(input, preds)
modelLSTM.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 35)               0         
 torization)                                                     
                                                                 
 embedding_2 (Embedding)     (None, 35, 100)           1604200   
                                                                 
 dropout_12 (Dropout)        (None, 35, 100)           0         
                                                                 
 bidirectional_6 (Bidirectio  (None, 35, 200)          160800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 200)              2408

In [35]:

from tensorflow.keras import layers
class DCNN(tf.keras.Model):

    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=50,
                 FFN_units=256,
                 nb_classes=4,
                 dropout_rate=0.1,
                 training=False,
                 name='dcnn'):
        super(DCNN,self).__init__(name=name)

        self.embeddings=layers.Embedding(vocab_size,emb_dim,embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),trainable=False)

        self.bigram=layers.Conv1D(filters=nb_filters,kernel_size=2,
                                  padding='valid',activation='relu')

        self.trigram=layers.Conv1D(filters=nb_filters,kernel_size=3,
                                  padding='valid',activation='relu')

        self.fourgram=layers.Conv1D(filters=nb_filters,kernel_size=4,
                                  padding='valid',activation='relu')   

        self.pooling=layers.GlobalMaxPool1D()

        self.dense_1=layers.Dense(units=FFN_units,activation='relu')
        self.dropout=layers.Dropout(rate=dropout_rate)


        self.dense_2=layers.Dense(units=nb_classes,activation='softmax')  

    def call(self,inputs,training):
        x=self.embeddings(inputs)
        x_1=self.bigram(x)
        x_1=self.pooling(x_1)
        #x_2=self.trigram(x)
        #x_2=self.pooling(x_2)
        #x_3=self.bigram(x)
        #x_3=self.pooling(x_3)

        merged=tf.concat([x_1],axis=-1)
        merged=self.dense_1(merged)
        merged=self.dropout(merged,training)
        output=self.dense_2(merged)

        return output

In [36]:
# x_train = vectorizer(np.array([[s] for s in X_train])).numpy()
# x_val = vectorizer(np.array([[s] for s in X_test])).numpy()
# x_virgin = vectorizer(np.array([[s] for s in X_virgin])).numpy()

y_train = np.array(y_train)
y_val = np.array(y_test)
y_virgin = np.array(y_virgin)

In [54]:
modelLSTM.compile(
    loss="sparse_categorical_crossentropy", optimizer=Adam(learning_rate=0.0005), metrics=["acc"]
)
modelLSTM.fit(X_train, y_train, batch_size=128, epochs=100, validation_data=(X_test, y_val),callbacks=[callback])

Epoch 1/100
152/152 [==============================] - 34s 172ms/step - loss: 1.2303 - acc: 0.5154 - val_loss: 0.9912 - val_acc: 0.6130
Epoch 2/100
152/152 [==============================] - 25s 166ms/step - loss: 0.9516 - acc: 0.6371 - val_loss: 0.8019 - val_acc: 0.6880
Epoch 3/100
152/152 [==============================] - 26s 174ms/step - loss: 0.8169 - acc: 0.6918 - val_loss: 0.6551 - val_acc: 0.7670
Epoch 4/100
152/152 [==============================] - 27s 175ms/step - loss: 0.6969 - acc: 0.7379 - val_loss: 0.5570 - val_acc: 0.8050
Epoch 5/100
152/152 [==============================] - 27s 174ms/step - loss: 0.5891 - acc: 0.7812 - val_loss: 0.4895 - val_acc: 0.8160
Epoch 6/100
152/152 [==============================] - 26s 174ms/step - loss: 0.5095 - acc: 0.8107 - val_loss: 0.3938 - val_acc: 0.8520
Epoch 7/100
152/152 [==============================] - 26s 173ms/step - loss: 0.4486 - acc: 0.8343 - val_loss: 0.3555 - val_acc: 0.8630
Epoch 8/100
152/152 [===========================

In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer=Adam(learning_rate=0.0009), metrics=["acc"]
)
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data=(X_test, y_val),callbacks=[callback])

Epoch 1/100
152/152 [==============================] - 3s 15ms/step - loss: 0.3173 - acc: 0.8790 - val_loss: 0.2785 - val_acc: 0.8970
Epoch 2/100
152/152 [==============================] - 2s 14ms/step - loss: 0.3016 - acc: 0.8846 - val_loss: 0.2951 - val_acc: 0.8890
Epoch 3/100
152/152 [==============================] - 2s 14ms/step - loss: 0.2898 - acc: 0.8893 - val_loss: 0.2747 - val_acc: 0.8950
Epoch 4/100
152/152 [==============================] - 2s 14ms/step - loss: 0.2902 - acc: 0.8870 - val_loss: 0.2608 - val_acc: 0.8900
Epoch 5/100
152/152 [==============================] - 2s 14ms/step - loss: 0.2838 - acc: 0.8893 - val_loss: 0.2551 - val_acc: 0.9000
Epoch 6/100
152/152 [==============================] - 2s 14ms/step - loss: 0.2705 - acc: 0.8948 - val_loss: 0.2544 - val_acc: 0.9000
Epoch 7/100
152/152 [==============================] - 2s 14ms/step - loss: 0.2606 - acc: 0.8962 - val_loss: 0.2830 - val_acc: 0.8950
Epoch 8/100
152/152 [==============================] - 2s 14ms

In [55]:
predicted_Y = modelLSTM.predict(X_virgin)
y_classes = predicted_Y.argmax(axis=-1)
y_classes
from sklearn import metrics
categories = ['happy','sadness','anger','fear','love']
print(metrics.classification_report(y_virgin, y_classes, target_names=categories))
test = ['I feel a bit sad today','Today, mom died','Jane and I argued yesterday, despite that ','I am not happy']

              precision    recall  f1-score   support

       happy       0.92      0.93      0.93        74
     sadness       1.00      0.94      0.97        69
       anger       0.84      1.00      0.91        16
        fear       1.00      0.97      0.98        31
        love       0.70      0.78      0.74         9

    accuracy                           0.94       199
   macro avg       0.89      0.92      0.91       199
weighted avg       0.94      0.94      0.94       199



In [ ]:
#modelLSTM.save('model_KaggleLSTM')

INFO:tensorflow:Assets written to: model_KaggleLSTM/assets


INFO:tensorflow:Assets written to: model_KaggleLSTM/assets


In [ ]:
# from keras import models
# model_2 = models.load_model('model_KaggleLSTM')

In [ ]:
predicted_Y = model.predict(X_virgin)
y_classes = predicted_Y.argmax(axis=-1)
y_classes
from sklearn import metrics
categories = ['happy','sadness','anger','fear','love']
print(metrics.classification_report(y_virgin, y_classes, target_names=categories))

              precision    recall  f1-score   support

       happy       0.96      0.88      0.92        74
     sadness       0.96      0.96      0.96        69
       anger       0.84      1.00      0.91        16
        fear       0.97      0.97      0.97        31
        love       0.67      0.89      0.76         9

    accuracy                           0.93       199
   macro avg       0.88      0.94      0.90       199
weighted avg       0.94      0.93      0.93       199



In [ ]:
from sklearn import metrics
categories = ['happy','sadness','anger','fear','love']

print(metrics.classification_report(y_virgin, y_classes, target_names=categories))

              precision    recall  f1-score   support

       happy       0.97      0.91      0.94        74
     sadness       0.99      0.96      0.97        69
       anger       0.80      1.00      0.89        16
        fear       0.97      0.97      0.97        31
        love       0.75      1.00      0.86         9

    accuracy                           0.94       199
   macro avg       0.89      0.97      0.92       199
weighted avg       0.95      0.94      0.95       199



In [1]:
test = ['I feel a bit sad today','Today, mom died','Jane and I argued yesterday, despite that ','I am not happy']
predicted_Y = model.predict(test)
y_classes = predicted_Y.argmax(axis=-1)
y_classes
# from sklearn import metrics
# categories = ['happy','sadness','anger','fear','love']
# print(metrics.classification_report(y_virgin, y_classes, target_names=categories))

NameError: name 'model' is not defined